In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, QuantileTransformer, PowerTransformer, RobustScaler

In [ ]:
import gdown

# Идентификатор файла
file_id = '17gyvy7Jad8r32CJBxRvimBpUr2HbPg2B'
# Формируем URL для скачивания
url = f'https://drive.google.com/uc?id={file_id}'

# Скачиваем файл
gdown.download(url, 'downloaded_file.ext', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=17gyvy7Jad8r32CJBxRvimBpUr2HbPg2B
To: /content/downloaded_file.ext
100%|██████████| 725k/725k [00:00<00:00, 9.67MB/s]


'downloaded_file.ext'

In [ ]:
car_manufacture = {
    'Maruti': 'India',
    'Nissan': 'Japan',
    'Hyundai': 'South Korea',
    'Renault': 'France',
    'Audi': 'Germany',
    'Mahindra': 'India',
    'Volkswagen': 'Germany',
    'Tata': 'India',
    'BMW': 'Germany',
    'Volvo': 'Sweden',
    'Honda': 'Japan',
    'Toyota': 'Japan',
    'Chevrolet': 'USA',
    'Fiat': 'Italy',
    'Ford': 'USA',
    'Skoda': 'Czech Republic',
    'Jeep': 'USA',
    'Mercedes-Benz': 'Germany',
    'Lexus': 'Japan',
    'Datsun': 'Japan',
    'Jaguar': 'UK',
    'Mitsubishi': 'Japan',
    'Force': 'India',
    'Ashok': 'India',
    'Peugeot': 'France',
    'Land': 'UK',
    'Ambassador': 'India',
    'Isuzu': 'Japan',
    'MG': 'UK',
    'Opel': 'Germany',
    'Daewoo': 'South Korea',
    'Kia': 'South Korea'
}
european_countries = {
    'Italy': 'Europe',
    'Czech Republic': 'Europe',
    'Sweden': 'Europe',
    'France': 'Europe',
    'Germany': 'Europe',
    'UK': 'Europe'
}
ordinal_owner = {'First Owner': 3, 'Second Owner': 2, 'Third Owner': 1,
       'Fourth & Above Owner': 0, 'Test Drive Car': 4}

In [ ]:
def iqr(my_array: np.array, i: float) -> tuple:
    Q1 = np.percentile(my_array, 25)
    Q3 = np.percentile(my_array, 75)

    # Вычисляем межквартильный размах
    IQR = Q3 - Q1

    # Определяем "интервал доверия"
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    if i > upper_bound:
      return upper_bound
    elif i < lower_bound:
      return lower_bound
    else:
      return i


In [ ]:
# Создаем датафрейм
df = pd.read_csv('/content/downloaded_file.ext')

# Создаем колонку производителя и бренда
df['brands'] = df.name.map(lambda i: i.split()[0])
df['manufactured'] = df.brands.map(lambda i: car_manufacture[i])
df.manufactured = df.manufactured.map(lambda i: european_countries[i] if i in european_countries else i)

# Используем ординальное кодирование для замены категоральной фичи
df.owner = df.owner.map(lambda i: ordinal_owner[i])

# Используем бинарное кодирование для замены категоральной фичи
columns = ['fuel','seller_type','transmission','manufactured']
for i in columns:
  df = pd.get_dummies(df, columns=[i])

# Чистим ДатаФрейм и готовим к обучению
df = df.drop(['brands', 'name'], axis = 1)
df.dropna(subset=['mileage(km/ltr/kg)'], inplace=True)

# Чистим колонку цены от аутлайеров
owner_4 = df[df.owner == 4]
df = df[df.selling_price < 1000000]
df = pd.concat([df, owner_4], ignore_index=True)

# Чистим колонку km_driven и mileage(km/ltr/kg) от аутлайеров с помощью IQR ведь именно эти данные могут через чур накручиваться
df.km_driven = df.km_driven.map(lambda i: iqr(df.km_driven, i))
df['mileage(km/ltr/kg)'] = df['mileage(km/ltr/kg)'].map(lambda i: iqr(df['mileage(km/ltr/kg)'], i))

# Находим NaN и меняем их на реальные значение в max_power
df.max_power[df.max_power == ' '] = '35'
df.max_power = df.max_power.astype('float64')

# Начинаем наше обучение
x = df.drop('selling_price', axis=1)
y = df['selling_price']

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

# Используем масштабирование QuantileTransformer ведь он подощёл лучше всего, и также используя эти цифры мы смогли достичь лучших результатов по mape и r2
pipeline = make_pipeline(
    QuantileTransformer(),
    PolynomialFeatures(degree = 3),
    Ridge(alpha = 0.4)
)

pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**.5
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')
print(f'MAPE: {mape}')

<ipython-input-28-657b10771306>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.max_power[df.max_power == ' '] = '35'


MAE: 70903.11785881764
MSE: 9549436603.347355
RMSE: 97721.21879790159
R2: 0.8203203438390669
MAPE: 0.21272874093926902


# **Validation**

In [ ]:

# Идентификатор файла
file_id = '1VqFHqaUMKfLNOZsLABshSjFWK6cRoJOE'
# Формируем URL для скачивания
url = f'https://drive.google.com/uc?id={file_id}'

# Скачиваем файл
gdown.download(url, 'downloaded_file2.ext', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1VqFHqaUMKfLNOZsLABshSjFWK6cRoJOE
To: /content/downloaded_file2.ext
100%|██████████| 102k/102k [00:00<00:00, 4.46MB/s]


'downloaded_file2.ext'

In [ ]:
# Создаем датафрейм
dfv = pd.read_csv('/content/downloaded_file2.ext')
dfv

,name,year,km_driven,fuel,seller_type,transmission,owner,mileage(km/ltr/kg),engine,max_power,seats,ID
0,Honda City 1.5 GXI,2004,110000,Petrol,Individual,Manual,Third Owner,12.80,1493.0,100.00,5.0,1
1,Tata Safari Storme EX,2014,291977,Diesel,Individual,Manual,First Owner,14.00,2179.0,138.10,7.0,2
2,Maruti Ritz VDi,2016,70000,Diesel,Individual,Manual,First Owner,23.20,1248.0,73.94,5.0,3
3,Honda City 2017-2020 VTEC,2006,120000,Petrol,Individual,Manual,Second Owner,16.90,1497.0,100.00,5.0,4
4,Maruti Swift VDI,2013,69000,Diesel,Individual,Manual,Second Owner,22.90,1248.0,74.00,5.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1022,Tata Indigo CS LS DiCOR,2010,70000,Diesel,Individual,Manual,Third Owner,15.70,1405.0,70.00,5.0,1023
1023,Tata Tigor 2017-2020 XZ Plus,2019,5200,Petrol,Individual,Manual,First Owner,20.30,1199.0,83.81,5.0,1024
1024,Honda Jazz 1.2 V i VTEC,2018,8576,Petrol,Dealer,Manual,First Owner,18.70,1199.0,88.70,5.0,1025
1025,Toyota Etios Liva G,2015,30000,Petrol,Individual,Manual,First Owner,17.71,1197.0,78.90,5.0,1026


In [ ]:
# Создаем датафрейм
dfv = pd.read_csv('/content/downloaded_file2.ext')

# Создаем колонку производителя и бренда
dfv['brands'] = dfv.name.map(lambda i: i.split()[0])
dfv['manufactured'] = dfv.brands.map(lambda i: car_manufacture[i])
dfv.manufactured = dfv.manufactured.map(lambda i: european_countries[i] if i in european_countries else i)

# Используем ординальное кодирование для замены категоральной фичи
dfv.owner = dfv.owner.map(lambda i: ordinal_owner[i])

# Используем бинарное кодирование для замены категоральной фичи
columns = ['fuel','seller_type','transmission','manufactured']
for i in columns:
  dfv = pd.get_dummies(dfv, columns=[i])

# Чистим ДатаФрейм и готовим к предсказанию
dfv = dfv.drop(['brands', 'name', 'ID'], axis = 1)

# Начинаем наше обучение
x = dfv
y_pred = pipeline.predict(x)

y_df = pd.DataFrame({'ID': range(1, len(y_pred)+1), 'price': y_pred})

y_df.to_csv('/content/predicted_prices.csv', index=False)